In [52]:
#데이터셋 불러오고 X, Y 정하기
import numpy as np
import pandas as pd

# ANN으로 한 번, 커널로 한 번, 트리나 xgb로 한 번, 다른 거 추가할 수 있으면 추가

df = pd.read_csv("/content/drive/MyDrive/Inhibitor classification/novdataset/Signaturize/PA_nov_signature_B2.csv", index_col=None)
X = df.iloc[:,1:-1].values
Y = df.iloc[:,-1].values

seed=0
np.random.seed(seed)

In [2]:
print(X.shape)
print(Y)

(115, 128)
[3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 1 0
 0 0 0 2]


In [53]:
#층화임의추출

from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)

for train_index, test_index in sss.split(X, Y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

TRAIN: [113  34  77  35 114   0  94 106  97  81   1  39  93  64   6 107  68 110
  63  11  69 103  27  28   9  18  43  87  40  98  71  26 104  16   4 111
  45  13  96  50  21  56  47  52  46  60  19 100   7  62  14  20  48  29
  82  57  89  44  88  38  70  72  33  67 102  86  42  91  25  24  66   5
  85  79 105  59  58  55  49 112  75  76  17  84  65   8  22  30  37  90
 109  99] TEST: [ 83  54   2  73  31  10  78  80  95  74 108  53  92  36  51  15  32  61
  23   3  41  12 101]


In [4]:
X_train.shape

(92, 128)

In [22]:
len(set(Y)) 

4

In [54]:
#X scaling
from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()

X_train_scaled = Scaler.fit_transform(X_train)
X_test_scaled = Scaler.transform(X_test)

In [69]:
#새로운 단일모델 짜보기
import keras
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Dropout

n_feat = X_train.shape[1] 
n_class = len(set(Y))  #Y_train.shape[1] #len(set(Y_mapped)) 

model = Sequential()
model.add(Dense(n_feat, input_dim=n_feat, activation='relu'))
model.add(Dense(n_feat, activation='relu'))
#model.add(Dropout(0.5)) # tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-07,amsgrad=False,name='Adam',**kwargs)
model.add(Dense(300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_class, activation='sigmoid'))
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=['accuracy'])

history = model.fit(X_train_scaled, Y_train, epochs=30, batch_size=5, validation_split=0.2)

Epoch 1/30
15/15 [==============================] - 1s 16ms/step - loss: 1.4056 - accuracy: 0.3836 - val_loss: 1.2143 - val_accuracy: 0.6316
Epoch 2/30
15/15 [==============================] - 0s 5ms/step - loss: 1.2498 - accuracy: 0.4932 - val_loss: 1.1232 - val_accuracy: 0.6842
Epoch 3/30
15/15 [==============================] - 0s 5ms/step - loss: 1.1526 - accuracy: 0.5068 - val_loss: 1.0764 - val_accuracy: 0.6316
Epoch 4/30
15/15 [==============================] - 0s 6ms/step - loss: 1.0825 - accuracy: 0.5068 - val_loss: 1.0626 - val_accuracy: 0.6842
Epoch 5/30
15/15 [==============================] - 0s 6ms/step - loss: 1.0263 - accuracy: 0.5890 - val_loss: 1.0474 - val_accuracy: 0.6316
Epoch 6/30
15/15 [==============================] - 0s 5ms/step - loss: 0.9741 - accuracy: 0.6027 - val_loss: 1.0289 - val_accuracy: 0.6316
Epoch 7/30
15/15 [==============================] - 0s 6ms/step - loss: 0.9244 - accuracy: 0.6438 - val_loss: 1.0286 - val_accuracy: 0.6316
Epoch 8/30
15/15 [=

In [ ]:
#과적합 너무 심함. 차원 축소? FS? 카테고리 축소?